In [10]:
import sys
sys.path.append('..')
import glob

from run_fft import FFTProcessor
fft_processor = FFTProcessor(method='fft', preprocess='none', value='norm', require_sid=True)

In [7]:
import rpy2
%load_ext rpy2.ipython

%R require("data.table")
%R require("ggplot2")
%R require("stringr")
%R require("fpp2") # https://github.com/robjhyndman/fpp2-package
# %R require("forecast")
# %R require("urca") # https://xiaorui.site/Forecasting-and-Time-Series-Methods/Lecture/5_ADF.html
# %R require("tseries") # This package provide adf.test()

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


1


A simple example to showcase the stationarity test

In [6]:
%%R 

# Example on a random data
x <- rnorm(1000)

defaultW <- getOption("warn")
options(warn = -1) # suppress warnings

adfpval <- adf.test(x) # Augmented Dickey-Fuller test

options(warn = defaultW) # restore warnings
print(adfpval)

# Expected output:
# Dickey-Fuller = (some value around -10), Lag order = 9, p-value = 0.01
# alternative hypothesis: stationary

# If warnings are not supressed, you will see
# In adf.test(x) : p-value smaller than printed p-value

# Conclusion: reject the null hypothesis, which means the series is stationary


	Augmented Dickey-Fuller Test

data:  x
Dickey-Fuller = -11.065, Lag order = 9, p-value = 0.01
alternative hypothesis: stationary



Prepare data for stationarity test

In [19]:
# Load BLOOM data

data_files_news_bloom = glob.glob('../data/bloom/news_split/*.nll')
nll_bloom_news = []
for file in data_files_news_bloom:
    nll = fft_processor._read_data(file)
    nll_bloom_news.extend(nll)
df_bloom_news = fft_processor._create_input_df(nll_bloom_news)

data_files_bloom_wiki = glob.glob('../data/bloom/wiki_split/*.nll')
nll_bloom_wiki = []
for file in data_files_bloom_wiki:
    nll = fft_processor._read_data(file)
    nll_bloom_wiki.extend(nll)
df_bloom_wiki = fft_processor._create_input_df(nll_bloom_wiki)

data_files_bloom_story = glob.glob('../data/bloom/story_split/*.nll')
nll_bloom_story = []
for file in data_files_bloom_story:
    nll = fft_processor._read_data(file)
    nll_bloom_story.extend(nll)
df_bloom_story = fft_processor._create_input_df(nll_bloom_story)

Conduct tests

In [25]:
%%R -i df_bloom_news -i df_bloom_wiki -i df_bloom_story

test_one_dt <- function(dt) {
  dt.test <- data.table(series_id = numeric(),
                        series_len = numeric(),
                        adfpval = numeric()
  )
  # Suppress warning
  defaultW <- getOption("warn")
  options(warn = -1)
  unique_series_ids <- unique(dt$sid)
  for (i in 1:length(unique_series_ids)) {
    s_id <- unique_series_ids[i]
    value <- dt[sid == s_id]$value
    if (length(value) < 10) {next}
    adfpval <- adf.test(value)$p.value # Augmented Dickey-Fuller test
    tmp <- data.table(series_id = s_id,
                      series_len = length(value),
                      adfpval = adfpval
    )
    dt.test <- rbindlist(list(dt.test, tmp))
    # slow, so print progress
    # if (i %% 500 == 0) {
    #   write(paste0("\rFinished ", i, " out of ", length(unique_series_ids)), stdout())
    # }
  }
  # Restore warning
  options(warn = defaultW)
  dt.test
}

# BLOOM 
dt.bloom.news <- data.table(df_bloom_news)
dt.bloom.news.test <- test_one_dt(dt.bloom.news)
print(nrow(dt.bloom.news.test[adfpval < 0.05]) / nrow(dt.bloom.news.test)) # This the proportion of series that pass the test

[1] 0.7672
